In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw06.ipynb")

<div class="alert alert-success" markdown="1">

#### Homework 6

# SQL, Regular Expressions, and GPTEECS

### EECS 398-003: Practical Data Science, Fall 2024

#### Due Thursday, October 17th at 11:59PM (due in **two** weeks)
    
</div>

## Instructions

Welcome to Homework 6! In this homework, you will practice writing SQL queries, use regular expressions to extract meaning out of messy text data, and apply your knowledge of cosine similarity and TF-IDF to implement a supercharged ChatGPT-like bot. See the [Readings section of the Resources tab on the course website](https://practicaldsc.org/resources/#readings) for supplemental resources.

You are given six slip days throughout the semester to extend deadlines. See the [Syllabus](https://practicaldsc.org/syllabus) for more details. With the exception of using slip days, late work will not be accepted unless you have made special arrangements with your instructor.

To access this notebook, you'll need to clone our [public GitHub repository](https://github.com/practicaldsc/fa24/). The [⚙️ Environment Setup](https://practicaldsc.org/env-setup) page on the course website walks you through the necessary steps. Once you're done, you'll submit your completed notebook to Gradescope.

Please start early and submit often. You can submit as many times as you'd like to Gradescope, and we'll grade your **most recent** submission.

<div class="alert alert-success" markdown="1">
Unlike other homeworks, Homework 6 <b>has no hidden tests</b>, because of its proximity to the Midterm Exam. This means the tests you see in your notebook are the exact same as the ones that will be used to grade your work on Gradescope. When you submit on Gradescope, you'll see your score shortly after you submit, once the autograder finishes running.
<br><br>
<b>Even though Homework 6 is due after the Midterm Exam, you should work on it before, since everything in the homework is in scope for the exam!</b> In particular, we recommend working on Questions 1-3 before the exam, because they provide core practice with SQL and regular expressions, both of which will appear on the exam. Question 4 is more "applied", and while cosine similarity, bag of words, and TF-IDF will appear on the exam, the best way to practice with those ideas is by working on relevant old exam problems at the <a href="https://study.practicaldsc.org"><b>study site</b></a>.
</div>

If you do fail a test in your notebook, look for a brief failure message that describes the error.

This homework is worth a total of **56 points**, all of which come from the autograder. The number of points each question is worth is listed at the start of each question. **The four questions in the assignment are independent, so feel free to move around if you get stuck**. Tip: if you're using Jupyter Lab, you can see a Table of Contents for the notebook by going to View > Table of Contents.

<!-- <a name='like-dataframe'>

</a>

<div class="alert alert-warning" markdown="1">
    
**Note**: Throughout this homework, you'll see statements like this frequently:

<blockquote>Complete the implementation of the function ____, which takes in a DataFrame <code>df</code> like <code>other_df</code> and _____.</blockquote>

What this means is that you should assume that `df` has the same number of columns as `other_df`, with the same column titles and data types, but potentially a different number of rows in a different order, with a potentially different index. You should always also assume that `df` has at least one row.

We have you implement functions like this to prevent you from hard-coding your answers to one specific dataset.

</div>
 -->
 
<div class="alert alert-danger" markdown="1">
<tt>for</tt>-loops are <strong>allowed</strong> throughout this entire homework.

</div>

To get started, run the **two** cells below, plus the cell at the top of the notebook that imports and initializes `otter`. The cell below installs a few new packages that weren't included in the `pds` conda environment that we'll need throughout the assignment.

In [2]:
!pip install duckdb
!pip install groq

In [3]:
import duckdb
import pandas as pd
import numpy as np
import os
import re
import groq
from IPython.display import Markdown

## Question 1: LoansQL 💵

---

In this question, you'll practice writing SQL queries involving the LendingClub loans dataset from [Lecture 7](https://practicaldsc.org/resources/lectures/lec07/lec07-filled.html) and Homework 4. Run the cell below to load in our dataset as the DataFrame `loans` and clean it using the same steps from lecture.

In [4]:
def clean_term_column(df):
    return df.assign(
        term=df['term'].str.split().str[0].astype(int)
    )

def clean_date_column(df):
    return (
        df
        .assign(date=pd.to_datetime(df['issue_d'], format='%b-%Y'))
        .drop(columns=['issue_d'])
    )

loans = (
    pd.read_csv('data/loans.csv')
    .pipe(clean_term_column)
    .pipe(clean_date_column)
)

loans

,id,loan_amnt,term,int_rate,grade,sub_grade,emp_title,verification_status,home_ownership,annual_inc,loan_status,purpose,desc,addr_state,dti,fico_range_low,fico_range_high,hardship_flag,mths_since_last_delinq,date
0,17965023,18000.0,60,16.99,D,D3,sales,Source Verified,RENT,60000.0,Charged Off,debt_consolidation,NaN,MN,23.22,700.0,704.0,N,72.0,2014-06-01
1,111414087,10000.0,36,16.02,C,C5,mechanic,Source Verified,OWN,50000.0,Current,home_improvement,NaN,IL,6.14,680.0,684.0,N,6.0,2017-06-01
2,95219557,12800.0,36,7.99,A,A5,general manager,Not Verified,MORTGAGE,47500.0,Fully Paid,debt_consolidation,NaN,MO,12.89,705.0,709.0,N,66.0,2016-12-01
3,142831837,16000.0,60,23.40,E,E1,nurse,Source Verified,MORTGAGE,120000.0,Current,home_improvement,NaN,FL,5.66,670.0,674.0,N,NaN,2018-10-01
4,140113255,40000.0,60,7.84,A,A4,staff pharmacist,Verified,MORTGAGE,150000.0,Current,debt_consolidation,NaN,MN,12.24,735.0,739.0,N,NaN,2018-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,16121830,20000.0,36,6.03,A,A1,senior buyer,Verified,RENT,124000.0,Fully Paid,debt_consolidation,NaN,CA,6.91,720.0,724.0,N,54.0,2014-05-01
6296,105091208,20000.0,36,7.99,A,A5,registered nurse,Source Verified,OWN,220000.0,Fully Paid,debt_consolidation,NaN,CA,5.66,695.0,699.0,N,NaN,2017-04-01
6297,63990101,10800.0,60,18.49,E,E2,technician,Verified,OWN,35000.0,Charged Off,debt_consolidation,NaN,NY,25.39,675.0,679.0,N,39.0,2015-11-01
6298,37641672,15000.0,60,14.31,C,C4,software engineer,Source Verified,RENT,150000.0,Fully Paid,credit_card,NaN,CA,10.63,660.0,664.0,N,22.0,2014-12-01


As we did in [Lecture 10](https://practicaldsc.org/resources/lectures/lec10/lec10-filled.html#SQL), we will use the Python module `duckdb` to execute SQL queries within our Jupyter Notebook. We've included code to install and import `duckdb` at the top of this notebook.

Specifically, we will use the `run_sql` function we defined in lecture.
- `run_sql` takes in a string containing a SQL query.
- `run_sql` outputs the result of running the query, treating all DataFrames mentioned in the query as if they were SQL tables. The result it outputs (here) is a **DataFrame**.

In [5]:
def run_sql(query_str):
    if query_str == ... or query_str.strip() == '':
        raise NotImplementedError('The input passed to run_sql is empty. Update it to include your query.')
    out = duckdb.query(query_str)
    return out.to_df()

For example, the following call to `run_sql` references `loans`, a DataFrame already defined in our notebook. It returns a new DataFrame (and doesn't modify the original `loans` DataFrame).

In [6]:
run_sql('''
SELECT * FROM loans
WHERE term = 60 AND loan_amnt > 20000
''')

,id,loan_amnt,term,int_rate,grade,sub_grade,emp_title,verification_status,home_ownership,annual_inc,loan_status,purpose,desc,addr_state,dti,fico_range_low,fico_range_high,hardship_flag,mths_since_last_delinq,date
0,140113255,40000.0,60,7.84,A,A4,staff pharmacist,Verified,MORTGAGE,150000.0,Current,debt_consolidation,None,MN,12.24,735.0,739.0,N,NaN,2018-09-01
1,143365202,30000.0,60,20.89,D,D4,graphics designer/ ui developer,Source Verified,RENT,80000.0,Current,credit_card,None,CA,29.34,700.0,704.0,N,17.0,2018-11-01
2,94378468,24000.0,60,21.49,D,D5,producer,Verified,RENT,85000.0,Charged Off,debt_consolidation,None,CA,15.92,660.0,664.0,N,29.0,2016-12-01
3,99330538,24000.0,60,23.99,E,E2,courier,Source Verified,MORTGAGE,80000.0,Fully Paid,debt_consolidation,None,GA,4.29,660.0,664.0,N,NaN,2017-02-01
4,59411554,32000.0,60,18.55,E,E2,patients care tech,Verified,OWN,75000.0,Fully Paid,debt_consolidation,None,CA,38.50,670.0,674.0,N,80.0,2015-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,16341922,20400.0,60,14.49,C,C4,information security engineer,Source Verified,OWN,133000.0,Fully Paid,home_improvement,None,GA,14.02,710.0,714.0,N,82.0,2014-05-01
490,137876020,40000.0,60,11.55,B,B4,police sergeant,Source Verified,MORTGAGE,150000.0,Current,debt_consolidation,None,IL,17.14,705.0,709.0,N,NaN,2018-08-01
491,78659466,30300.0,60,29.96,G,G4,rn,Verified,MORTGAGE,70000.0,Charged Off,debt_consolidation,None,IN,21.93,680.0,684.0,N,NaN,2016-05-01
492,56109274,21000.0,60,20.99,E,E5,mailhandler,Verified,RENT,54000.0,Fully Paid,debt_consolidation,None,NY,28.53,665.0,669.0,N,19.0,2015-08-01


The above query happens to be equivalent to the following:

In [7]:
loans[(loans['term'] == 60) & (loans['loan_amnt'] > 20000)]

,id,loan_amnt,term,int_rate,grade,sub_grade,emp_title,verification_status,home_ownership,annual_inc,loan_status,purpose,desc,addr_state,dti,fico_range_low,fico_range_high,hardship_flag,mths_since_last_delinq,date
4,140113255,40000.0,60,7.84,A,A4,staff pharmacist,Verified,MORTGAGE,150000.0,Current,debt_consolidation,NaN,MN,12.24,735.0,739.0,N,NaN,2018-09-01
35,143365202,30000.0,60,20.89,D,D4,graphics designer/ ui developer,Source Verified,RENT,80000.0,Current,credit_card,NaN,CA,29.34,700.0,704.0,N,17.0,2018-11-01
48,94378468,24000.0,60,21.49,D,D5,producer,Verified,RENT,85000.0,Charged Off,debt_consolidation,NaN,CA,15.92,660.0,664.0,N,29.0,2016-12-01
52,99330538,24000.0,60,23.99,E,E2,courier,Source Verified,MORTGAGE,80000.0,Fully Paid,debt_consolidation,NaN,GA,4.29,660.0,664.0,N,NaN,2017-02-01
54,59411554,32000.0,60,18.55,E,E2,patients care tech,Verified,OWN,75000.0,Fully Paid,debt_consolidation,NaN,CA,38.50,670.0,674.0,N,80.0,2015-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6219,16341922,20400.0,60,14.49,C,C4,information security engineer,Source Verified,OWN,133000.0,Fully Paid,home_improvement,NaN,GA,14.02,710.0,714.0,N,82.0,2014-05-01
6224,137876020,40000.0,60,11.55,B,B4,police sergeant,Source Verified,MORTGAGE,150000.0,Current,debt_consolidation,NaN,IL,17.14,705.0,709.0,N,NaN,2018-08-01
6234,78659466,30300.0,60,29.96,G,G4,rn,Verified,MORTGAGE,70000.0,Charged Off,debt_consolidation,NaN,IN,21.93,680.0,684.0,N,NaN,2016-05-01
6275,56109274,21000.0,60,20.99,E,E5,mailhandler,Verified,RENT,54000.0,Fully Paid,debt_consolidation,NaN,NY,28.53,665.0,669.0,N,19.0,2015-08-01


All of the questions below will ask you to assign your answer to a **string**. To test your code, we will call `run_sql` on the string that you define, and make sure the resulting DataFrame has the right properties. We suggest you use multi-line strings (defined by `'''` triple quotes `'''`) like in the example call to `run_sql` above.

Most of the syntax you need to answer the queries here was covered in Lecture 10. The chart at the start of the [SQL section of Lecture 10](https://practicaldsc.org/resources/lectures/lec10/lec10-filled.html#SQL) contains a nice summary of the necessary keywords.

### Question 1.1 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Assign `query_1` to a string containing a SQL query that finds **the total (sum) of all loan amounts for each loan purpose, only among loans given in Michigan (`'MI'`)**. The DataFrame that results from calling `run_sql(query_1)` should have two columns, **`'purpose'`** and **`'total_loans'`**, and should be sorted in **descending order** of `'total_loans'`.

In [8]:
query_1 = '''
    SELECT purpose, SUM(loan_amnt) AS total_loans
    FROM loans 
    WHERE addr_state = 'MI'
    GROUP BY purpose
    ORDER BY total_loans DESC
'''
run_sql(query_1)

,purpose,total_loans
0,debt_consolidation,1541525.0
1,credit_card,582675.0
2,home_improvement,113375.0
3,other,110275.0
4,house,68600.0
5,major_purchase,45800.0
6,small_business,25900.0
7,moving,6400.0


In [9]:
grader.check("q01_01")

q01_01 results: All test cases passed!

### Question 1.2 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Assign `query_2` to a string containing a SQL query that finds **the average credit score per state, among states _having_ at least 150 loans**. The DataFrame that results from calling `run_sql(query_2)` should have two columns, **`'state'`** and **`'average_credit'`**, and should be sorted in **increasing order** of `'average_credit'`.

Some guidance:
- Extract credit scores from the `'fico_range_low'` column in `loans`.
- One of the SQL keywords you need to use was _italicized_ in the first sentence above.

In [10]:
query_2 = '''
    SELECT addr_state AS state , AVG(fico_range_low) AS average_credit
    FROM loans
    GROUP BY addr_state
    HAVING COUNT(loan_amnt) > 150
    ORDER BY average_credit ASC
'''
run_sql(query_2)


,state,average_credit
0,FL,695.130952
1,VA,696.338798
2,NJ,696.710526
3,OH,697.401961
4,MI,697.672414
5,CA,698.154696
6,MD,698.245033
7,IL,698.582375
8,GA,698.712121
9,PA,698.731707


In [11]:
grader.check("q01_02")

q01_02 results: All test cases passed!

### Question 1.3 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

The LendingClub uses **simple interest** to calculate loan payments. Using simple interest, the total amount due for a loan is:

$$\text{Total Amount Due} = \text{Loan Amount} \cdot \left(1 + (\text{Interest Rate} \cdot \text{Loan Length in Years}) \right)$$

For example, a loan for \\$10,000 at an interest rate of 15% for 5 years would pay a total amount of \\$17,500:

$$10000 \cdot (1 + 0.15 \cdot 5) = 17500$$

Since there are 60 months in 5 years, this lendee would pay $\frac{\$17,500}{60} = \$291.67$ per month for 60 months. **Note that the percentage 15% is equivalent to the decimal 0.15.**

More generally:

$$\text{Monthly Payments} = \frac{\text{Total Amount Due}}{\text{Loan Length in Months}} = \frac{\text{Loan Amount} \cdot \left(1 + (\text{Interest Rate} \cdot \text{Loan Length in Years}) \right)}{\text{Loan Length in Months}}$$

Assign `query_3` to a string containing a SQL query that finds **loan amount, term, interest rate, and monthly payment amount of the single loanholder with the highest monthly payments**. The DataFrame that results from calling `run_sql(query_3)` should have four columns, `'amount'`, `'term'`, `'interest'`, and `'monthly'`, and should only have a **single row**.

In [12]:
query_3 = '''
SELECT loan_amnt AS amount, term, int_rate AS interest, (loan_amnt * ((1+(interest/100)*(term/12)))/term) AS monthly 
FROM loans
ORDER BY monthly DESC
LIMIT 1;
'''
run_sql(query_3)

,amount,term,interest,monthly
0,39475.0,36,23.88,1882.080278


In [13]:
grader.check("q01_03")

q01_03 results: All test cases passed!

## Question 2: Practice with Regular Expressions 📕

---

Regular expressions can be tricky, and the best way to gain familiarity with them is through lots of practice.

In this question, you will work through 10 parts, **each of which requires you to write a regular expression that matches strings that satisfy certain criteria**. You will do this by – as usual – completing the implementation of a function. In Questions 2.1 through 2.9, your function will take in a string and return `True` if the string follows the pattern and `False` otherwise.

- Make sure to take a close look at the examples for each function, as they provide useful guidance for the types of strings you should and shouldn't match.
- Make sure to refer to the [Regular Expression Resources](https://practicaldsc.org/resources/#regular-expressions) on the course website. In particular, we recommend having [regex101.com](https://regex101.com/) open while working, along with the [cheat sheet](https://practicaldsc.org/resources/other/berkeley-regex-reference.pdf).
- The number of points each part is worth tells you its relative difficulty level – some are worth <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div> and some are worth <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>. If you're spending lots of time on exercises worth <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>, take a close look at the syntax from [Lecture 9](https://practicaldsc.org/resources/lectures/lec09/lec09-filled.html), as there is probably an easier way of writing the necessary pattern!
- The 10 parts are all independent, and are **not** sorted by difficulty – some of the easiest parts are in the middle or towards the end!

### Question 2.1  <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that have `'['` as the third character and `']'` as the sixth character. Example behavior is given below.

```python
>>> match_1("abcde]")
False

>>> match_1("ab[cde")
False

>>> match_1("ab[cd]ef")
True
```

In [14]:
def match_1(string):
    pattern = r'^..\[..\].*$'

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [15]:
grader.check("q02_01")

q02_01 results: All test cases passed!

### Question 2.2 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that are phone numbers that start with `'(734)'` and follow the format `'(xxx) xxx-xxxx'` (`'x'` represents a digit). Example behavior is given below.

```python
>>> match_2("(734) 456-7890")
True

>>> match_2("(123) 456-7890")
False

>>> match_2("(734) 456-7890b")
False
```

Note that there is a space between `'(xxx)'` and `'xxx-xxxx'`!

In [16]:
def match_2(string):
    pattern = r'^\(734\) \d{3}-\d{4}$'

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [17]:
grader.check("q02_02")

q02_02 results: All test cases passed!

### Question 2.3 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that:
- are between 6 and 10 characters long (inclusive),
- contain only alphanumeric characters, whitespace and `'?'`, and
- end with `'?'`.

Example behavior is given below.

```python
>>> match_3('qw?ertsd?')
True

>>> match_3("ab   c ?")
True

>>> match_3(" adf!qes ?")
False

>>> match_3('wwwWW .? ')
False
```

Note that `'?'` is a special character.

In [18]:
def match_3(string):
    pattern = r"^[A-Za-z \?]{5,9}\?$"

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [19]:
grader.check("q02_03")

q02_03 results: All test cases passed!

### Question 2.4 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Write a regular expression that matches strings with exactly two `'$'`, one of which is at the start of the string, such that:
- the characters between the two `'$'` can be anything (including nothing) except the lowercase letters `'a'`, `'b'`, and `'c'`, (and `'$'`), and
- the characters after the second `'$'` can only be the **lowercase or uppercase** letters `'a'`/`'A'`, `'b'`/`'B'`, and `'c'`/`'C'`, with every `'a'`/`'A'` before every `'b'`/`'B'`, and every `'b'`/`'B'` before every `'c'`/`'C'`. There **must be** at least one `'a'` or `'A'`, at least one `'b'` or `'B'`, and at least one `'c'` or `'C'`.

Example behavior is given below.

```python
>>> match_4("$!@#$aABc")
True

>>> match_4('$qw!!  $aaBC')
True

>>> match_4('$a$aABc')
False

>>> match_4('$!@$')
False
```

In [20]:
def match_4(string):
    pattern = r"^\$([^abc]*)\$[aA]+[bB]+[cC]+"

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [21]:
grader.check("q02_04")

q02_04 results: All test cases passed!

### Question 2.5 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that represent valid Python file names, including the extension. For simplicity, assume that file names only contain letters, numbers, and underscores. Example behavior is given below.

```python
>>> match_5("eecs398.py")
True

>>> match_5('eecs398_.py')
True

>>> match_5("here is a Python file eecs398.py")
False

>>> match_5("eecs398+.py")
False
```

In [22]:
def match_5(string):
    pattern = r"^[\w]+\.py$"

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [23]:
grader.check("q02_05")

q02_05 results: All test cases passed!

### Question 2.6 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that:
- are made up of only lowercase letters and exactly one underscore (`'_'`), and
- have at least one lowercase letter on both sides of the underscore.

Example behavior is given below.

```python
>>> match_6("aab_cbbbc")
True

>>> match_6("zebra_d")
True

>>> match_6("aab_Abbbc")
False

>>> match_6("zebra_")
False
```

In [24]:
def match_6(string):
    pattern = r"^[a-z]+_[a-z]+$"

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [25]:
grader.check("q02_06")

q02_06 results: All test cases passed!

### Question 2.7 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that start with and end with an underscore (`'_'`). Example behavior is given below.

```python
>>> match_7("_abc_")
True

>>> match_7("_ZeBr@45Din000!!!\b_")
True

>>> match_7("abc")
False

>>> match_7("_ncde")
False

>>> match_7("_") # Need at least two underscores!
False
```

In [26]:
def match_7(string):
    pattern = r"^_.+_$"

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [27]:
grader.check("q02_07")

q02_07 results: All test cases passed!

### Question 2.8 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Apple serial numbers are strings of length 1 or more that are made up of any characters, other than
- the uppercase letter `'O'`, 
- the lowercase letter `'i`', and 
- the number `'1'`.

Write a regular expression that matches strings that are valid Apple serial numbers. Example behavior is given below.

```python
>>> match_8('ASDJKL9380JKAL')
True

>>> match_8("ASJDKLFK0ASDo!!!!!!! !!!!!!!!!")
True

>>> match_8('iPhone 10')
False

>>> match_8("hi ASDJKL9380JKAL")
False
```

In [28]:
def match_8(string):
    pattern = r'^[^Oi1]+$'

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [29]:
grader.check("q02_08")

q02_08 results: All test cases passed!

### Question 2.9 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Suppose DataID numbers are formatted as `'SC-NN-CCC-NNNN'`, where 
- SC represents state code in uppercase (e.g. `'MI'`),
- NN represents a number with 2 digits (e.g. `'98'`),
- CCC represents a three letter city code in uppercase (e.g. `'DTW'`), and
- NNNN represents a number with 4 digits (e.g. `'1998'`).

Write a regular expression that matches strings that are DataID numbers corresponding to the cities of `'DTW'` (Detroit) or `'LAN'` (Lansing), or the state of `'TX'` (Texas). Assume that there is only one city named `'DTW'` and only one city named `'LAN'`.

Example behavior is given below.

```python
>>> match_9('MI-32-LAN-1232')
True

>>> match_9('TX-32-DTW-1232')
True

# Lansing is not in California!
>>> match_9('CA-32-LAN-1232')
False

>>> match_9('mI-32-LAN-1232')
False
```

In [30]:
def match_9(string):
    pattern = r"^[MITX]{2}-\d{2}-[A-Z]{3}-\d{4}$"

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [31]:
grader.check("q02_09")

q02_09 results: All test cases passed!

### Question 2.10 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

In this final part, your task involves more than writing a single regular expression.

Complete the implementation of the function `match_10`, which takes in a string (`string`) and:
- converts the string to lowercase,
- removes all non-alphanumeric characters (i.e. removes everything that is not in the `\w` character class), and the letter `'a'`, and
- returns a list of every **non-overlapping** three-character substring in the remaining string, starting from the beginning of the string.
   
Example behavior is given below.

```python
>>> match_10('Ab..DEF')
['bde']

>>> match_10('FINALS are COMING A')
['fin', 'lsr', 'eco', 'min']

>>> match_10('h9i9hOWW44areY@')
['h9i', '9ho', 'ww4', '4re']
```

Here's how `match_10` should process `'Ab..DEF'`:

1. Convert to lowercase: `'ab..def'`.
2. Remove non-alphanumeric characters and the letter `'a'`: `'bdef'`.
3. Starting from the beginning of the string, there is only a single non-overlapping three character substring: `'bde'`. Hence, we return `['bde']`.

Some guidance: 
- Perform your operations in the exact order described above, otherwise your code may not pass all the tests.
- Don't use a `for`-loop. You'll need to use `re.sub` in addition to `re.findall`.

In [32]:
def match_10(string):
    string = string.lower()
    clean_string = re.sub(r'[^a-zA-Z0-9]|a| ', '', string)
    clean_string
    out = []
    for i in range(0,(len(clean_string)-2),3):
        out.append(clean_string[i:i+3])
    return out
    
    

# Feel free to change the input below to test out your implementation of match_10.
#match_10('FINALS are COMING A')
match_10('FINALS are COMING A')

['fin', 'lsr', 'eco', 'min']

In [33]:
grader.check("q02_10")

q02_10 results: All test cases passed!

## Question 3: Capture Groups 📡

---

The dataset stored in `data/messy.txt` contains personal information from a fictional website that a user scraped from web server logs. Within this dataset, there are four fields that are of interest to you:
1. Social Security Numbers
1. Bitcoin Addresses
1. Email Addresses 
1. Street Addresses

Your job is to use `re.findall` to extract out the relevant pieces of information from `messy.txt`. **Since this data is very messy, your function will be allowed to miss ~5% of the records in each list. Good spot checking using certain useful substrings (e.g. `'@'` for emails) should help assure correctness!** As usual, your functions will be tested on a sample of the file `messy.txt`.

Note that there are multiple "delimiters" (separators) in use in the file; there are few enough of them that you can safely determine what they are. **Before attempting any of the parts here, open `data/messy.txt` in your favorite text editor and explore!**

### Question 3.1 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Complete the implementation of the function `extract_ssns`, which takes in a string (`string`) containing the contents of a server log file and returns the Social Security Numbers in the file as a list. Example behavior is given below.

```python
>>> extract_ssns('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
['423-01-9575']

>>> out = extract_ssns(open('data/messy.txt', encoding='utf8').read())
>>> out[0]
'380-09-9403'
```

Some guidance:
- For our purposes, an SSN is a string of the form 3 digits-2 digits-4 digits.
- The returned list should not contain any empty strings or the string `'null'`.

In [34]:
def extract_ssns(string):
    pattern = r'\d{3}-\d{2}-\d{4}'
    return re.findall(pattern, string)
    
# To test your work, first run:
extract_ssns('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
# Then, once that works, uncomment:
# extract_ssns(open('data/messy.txt', encoding='utf8').read())

['423-01-9575']

In [35]:
grader.check("q03_01")

q03_01 results: All test cases passed!

### Question 3.2  <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Complete the implementation of the function `extract_bitcoin_addresses`, which takes in a string (`string`) containing the contents of a server log file and returns the Bitcoin addresses in the file as a list. Example behavior is given below. 

```python
>>> extract_bitcoin_addresses('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
['1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2']

>>> out = extract_bitcoin_addresses(open('data/messy.txt', encoding='utf8').read())
>>> out[0]
'18A8rBU3wvbLTSxMjqrPNc9mvonpA4XMiv'
```

Some guidance:
- Assume Bitcoin addresses are alphanumeric strings.
- The returned list should not contain any empty strings or the string `'null'`.

In [36]:
def extract_bitcoin_addresses(string):
    pattern = r'in:[A-Za-z0-9]+'
    word = re.findall(pattern, string)
    word = [i[3:] if i != "in:null" else ''  for i in word]
    if '' in word:
        word.remove('')
    return word
    
# To test your work, first run:
extract_bitcoin_addresses('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')

# Then, once that works, uncomment:
extract_bitcoin_addresses(open('data/messy.txt', encoding='utf8').read())

['18A8rBU3wvbLTSxMjqrPNc9mvonpA4XMiv',
 '1EB7kYpnfJSqS7kUFpinsmPF3uiH9sfRf1',
 '1E5fev4boabWZmXvHGVkHcNJZ2tLnpM6Zv',
 '1DqG3WcmGw74PjptjzcAmxGFuQdvWL7RCC',
 '1LfacbqCA7NZVq2u5CTTZsoEtncYuBWvNX',
 '1DoWjNx6PGwPvMQ5P7zeLWg6hnbGC184td',
 '1FLuKwdTrCiMrdS4wmRKCJfTRDZsq5f2q2',
 '1DY46TrozvX1SLvGH39Nc2kfx236gaAoLA',
 '17fo1ZbViKPFGiPXhNsAU8NUyw5bynRxDX',
 '17ZrXccNfAdyVt7DTqWDXKgdhmYtYKnmNN',
 '18SWMUwCnwHxoFAr2czJRsSBQgF8oCWDTu',
 '1PrAha8iCktRM2J67VXivQucjzvsTRwsd8',
 '1BovwmzX7oLyNZX76LqVy5SJKQLBYA4d7c',
 '1DAb3G5vbWAotDZrdQaFu1NYCTsEEgxWpM',
 '1L7UMt4HmedHLeowz7TAmTtgPKGzTHM7H3',
 '1CaBYBD2Hz21NHpQ266iVxZ2ZKjS29XvCp',
 '1NsK6BVxEPwxXLMjLkVkVLQzbjqLLEyT1i',
 '1AyiBohjtgByzouVbpCp2LqV2qATy5Nvgt',
 '1WtiMLn5TYheBkLQ6D3Vr4TjmkyR8aNLY',
 '17LwuggjFmVWYwDLKJ2zrxKrs6d21FMvij',
 '1FTXfdD4ZDsBu1ht3KvkPEd5GKFhTj9Pri',
 '13WNKycmFaQvbyARgMrvn434kmDRQQWzr2',
 '1LMRBCHL4Hi4aQR8nMNAYErrBVGLjXX5dv',
 '1KVfRPSwDTbAu9feKFSqFBap5swajp56kn',
 '13HcgY6Vn7oFPAzYnARVtZQS4GDbb7sfqg',
 '1GyFrbAJwcGQugdPBs7ecdf1

In [37]:
grader.check("q03_02")

q03_02 results: All test cases passed!

### Question 3.3 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Complete the implementation of the function `extract_emails`, which takes in a string (`string`) containing the contents of a server log file and returns the email addresses in the file as a list. Example behavior is given below.

```python
>>> extract_emails('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
['test@test55.umich.edu']

>>> out = extract_emails(open('data/messy.txt', encoding='utf8').read())
>>> out[0]
'dottewell0@gnu.org'
```

Some guidance:
- Assume that the usernames and domain names in an email address are alphanumeric. Domain names don't need to end in `'.com'` – assume that all parts of a domain name, including the very end, can be made up of any alphanumeric characters.
- The returned list should not contain any empty strings or the string `'null'`. (It likely won't by default, but we've included this instruction in all four parts of this question.)

In [38]:
def extract_emails(string):
    pattern  = r'\w+@[^\,|*(\t)]+'
    word = re.findall(pattern, string)
    if '' in word:
        word.remove('')
    return word
    
# To test your work, first run:
#extract_emails('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
# Then, once that works, uncomment:
extract_emails(open('data/messy.txt', encoding='utf8').read())

['dottewell0@gnu.org',
 'bassiter1@sphinn.com',
 'dtitmarsh2@dailymail.co.uk',
 'mcolliber3@fda.gov',
 'ohachard4@bbb.org',
 'aaikman5@cnet.com',
 'bgiovannacci6@theglobeandmail.com',
 'tbritton7@nytimes.com',
 'dcheeney8@mail.ru',
 'kkordes9@prweb.com',
 'bcottrilla@gmpg.org',
 'sdoumencb@cbslocal.com',
 'fandreec@nhs.uk',
 'epaviourd@ameblo.jp',
 'wlongegae@disqus.com',
 'bofihilyf@house.gov',
 'slinceg@xrea.com',
 'kdumphreyh@hc360.com',
 'mhowseleei@vistaprint.com',
 'gseelj@reuters.com',
 'lcroixk@ocn.ne.jp',
 'wudalll@engadget.com',
 'wmcdavittm@comsenz.com',
 'nalmondn@reference.com',
 'cjesseo@mail.ru',
 'mlibbeq@telegraph.co.uk',
 'jmacgorleyr@rediff.com',
 'cgooks@shinystat.com',
 'tdipplet@apple.com',
 'lwarinu@shop-pro.jp',
 'slethbridgev@java.com',
 'lteggartw@dion.ne.jp',
 'ecoulstonx@networkadvertising.org',
 'scheccuzziy@umn.edu',
 'ddunstanz@cnet.com#TTt34W5xdc',
 'fmcgooch11@joomla.org',
 'kgommowe13@tiny.cc',
 'sgoulding14@cmu.edu',
 'jgrelak15@blogs.com',
 'hcuddeha

In [39]:
grader.check("q03_03")

q03_03 results: All test cases passed!

### Question 3.4 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Complete the implementation of the function `extract_street_addresses`, which takes in a string (`string`) containing the contents of a server log file and returns the street addresses in the file as a list. Example behavior is given below.

```python
>>> extract_street_addresses('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
['530 High Street']

>>> out = extract_street_addresses(open('data/messy.txt', encoding='utf8').read())
>>> out[0]
'814 Monterey Court'
```

As before, the returned list should not contain any empty strings or the string `'null'`.

In [40]:
def extract_street_addresses(string):
    pattern = r'\d+ \w+ \w+'
    word = re.findall(pattern, string)
    return word

# To test your work, first run:
#extract_street_addresses('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
# Then, once that works, uncomment:
extract_street_addresses(open('data/messy.txt', encoding='utf8').read())

['814 Monterey Court',
 '62 Hooker Park',
 '27811 Clyde Gallagher',
 '32553 Riverside Pass',
 '17157 Clemons Alley',
 '75 Dorton Parkway',
 '5 Manitowish Trail',
 '84 Northland Center',
 '64404 Sundown Street',
 '47144 Mockingbird Street',
 '413 Sutherland Court',
 '8906 Almo Lane',
 '34223 Graceland Crossing',
 '94 Mcguire Lane',
 '1 Delladonna Circle',
 '169 Claremont Point',
 '19475 Meadow Vale',
 '147 Cascade Center',
 '762 Knutson Terrace',
 '301 Monument Trail',
 '39 Buena Vista',
 '94 Barby Circle',
 '98 Hansons Road',
 '83525 Calypso Way',
 '61 Heath Street',
 '60 Bay Hill',
 '923 Lindbergh Place',
 '1 Elmside Circle',
 '34 High Crossing',
 '63 Bonner Lane',
 '67 Superior Terrace',
 '846 Acker Road',
 '4 Menomonie Avenue',
 '858 Arrowood Terrace',
 '2566 Anhalt Trail',
 '30840 Sherman Hill',
 '29170 Parkside Lane',
 '150 Logan Street',
 '74 North Court',
 '51374 8th Hill',
 '6432 Continental Avenue',
 '46916 Donald Court',
 '1950 Beilfuss Hill',
 '949 Park Meadow',
 '38 Tennyso

In [41]:
grader.check("q03_04")

q03_04 results: All test cases passed!

## Question 4: GPTEECS 🤖

---

### Overview

Large Language Models (LLM), like GPT-4 by OpenAI, Claude by Anthropic, or Llama by Meta, are statistical models that were trained on massive datasets for the purpose of generating useful new text. [ChatGPT](https://chat.openai.com) and other similar chat interfaces make calls to an LLM API under-the-hood, and show you the results in a text message-like format.

Open ChatGPT or your favorite other LLM chat interface, and ask it:

> What's the difference between the late submission policy in EECS 467 and EECS 492?

Until very recently (when ChatGPT started being able to search the internet), ChatGPT would tell you that it doesn't know what EECS 467 and EECS 492 are. And even if it did give you an answer, it's not necessarily clear whether it pulled the answer from a reliable source, or whether it's still true today (it may have found syllabi online from many years ago, and could be hallucinating). 

### Retrieval-Augmented Generation (RAG)

A solution to this issue is **Retrieval-Augmented Generation (RAG)**. **In this question, we will use RAG to implement GPTEECS, a chat interface designed to answer questions about EECS syllabi.** Here's the general idea behind RAG, and how we'll use it in this question:

1. We want to implement a chat bot that can answer questions about something specific.<br><small>**Here**, we want our chat bot to answer questions about EECS class' syllabi.</small>
1. To do so, we download and store documents that contain the relevant context that we wish our LLM knew about.<br><small>**Here**, we'll download the syllabi of various EECS classes and store them as `.txt` files. We've already done this for you.</small>
1. Then, when the user asks a question – called a **query** – we determine which of our locally-stored documents are most relevant in answering their question.<br><small>**Here**, when a user asks a question about EECS class(es), we'll determine which syllabus documents are most likely to have the answer.</small>
1. Once we find the most relevant documents, we send the user's query, **along with** the most relevant documents, to our language model, allowing it to find the answer for us with the context it needs.

<center><img src="imgs/retrieval-augmented-generation.png" width=700><br>(<a href="https://towhee.io/tasks/detail/pipeline/retrieval-augmented-generation">image source</a>)</center>

RAG enables organizations to create customized chat interfaces that are better equipped to answer questions about the organization than an out-of-the-box language model. For instance, if you operated a store and wanted an AI-powered customer support chat, you may use RAG to create a chat bot that knows about your store's catalog, return policies, etc. ChatGPT even allows you to make custom GPTs [yourself](https://openai.com/index/introducing-gpts/) by uploading customized knowledge bases, and these (likely) use a process similar to RAG.

### FAQs

- **How do we determine which documents are most relevant to the user's query?** Here, we'll implement this using TF-IDF and cosine similarity, as we've seen in [Lecture 12](https://practicaldsc.org/resources/lectures/lec12/lec12-filled.html)! In practice, more sophisticated, state-of-the-art techniques for converting text to numbers are used (if you're curious, look into "word embeddings").
- **Why not just send all of the documents to our language model, instead of finding the documents that are most relevant?** LLMs have a [context window](https://www.hopsworks.ai/dictionary/context-window-for-llms), which is a limit on the length of the input query they can take in. If your query is too long, an LLM may not be able to process it. (And, if it includes unnecessary information, it can be hard for the LLM to give you an accurate response.)

### Your Task
The folder `data/syllabi` contains syllabi for several EECS classes. These documents together comprise our **corpus**.

In [42]:
!ls data/syllabi

183.txt 280.txt 373.txt 390.txt 465.txt 471.txt 481.txt 484.txt 489.txt 493.txt
203.txt 281.txt 376.txt 445.txt 467.txt 473.txt 482.txt 485.txt 490.txt 494.txt
270.txt 370.txt 388.txt 453.txt 470.txt 475.txt 483.txt 487.txt 492.txt


Shortly, using the ideas from Lecture 12, you will develop a working implementation of the following function:

```python
>>> top_n_similar_documents('C++ programming and systems design', 4, bow)
['482.txt', '473.txt', '370.txt', '281.txt']
```

And even cooler, you'll implement a function that can fully answer questions, like:

```python
>>> ask_gpteecs("I really want to learn theoretical probability and math, what should I take?")
'Based on your interest in theoretical probability and math, I recommend taking EECS 445: Introduction to Machine Learning. This course covers the foundational algorithms and "tricks of the trade" in machine learning, including regression, classification...'
```

### Question 4.1 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">1 Point</div>

First, let's figure out how to call a Large Language Model directly from our notebook.

OpenAI does have a Python API, but it's relatively limited on the free plan. Instead, we'll use tools from [Groq](https://groq.com/). Groq is a hardware company designing processors for training LLMs efficiently, and allows for fast, free access to open-source LLM APIs. We'll use the [Groq API](https://console.groq.com/docs/quickstart) to make calls to Meta's Llama 3 API. (As mentioned at the start of this section, Llama is Meta's competitor to GPT. So technically, we're not implementing GPTEECS, but EECSLlama?)

Go [**here**](https://console.groq.com/docs/quickstart) and create a Groq API key. Then, complete the implementation of `query_llama`, a function that takes in a string (`query_string`) and returns the text response that results from passing `query_string` to Groq. The function has largely been implemented for you; most of what you need to do is create an API key and put it in the right place below.

(Yes, confusingly, we're using the word "query" in this homework to refer to slightly different, but related, ideas: in SQL, queries are used to extract information from a **database**, and here, our queries pull information from an API.)

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

In [43]:
def query_llama(query_string):
    client = groq.Groq(
        api_key= ""
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query_string
            }
        ],
        model="llava-v1.5-7b-4096-preview",
        # temperature=0 # Try uncommenting this and running the call to query_llama below many times. What do you notice? Recomment it out afterwards.
    )

    return chat_completion.choices[0].message.content

# Feel free to change the input below to test out your implementation of query_llama.
# The Markdown function behaves like the print function,
# but renders text formatting (e.g. bolding, bullet points) when the output from Llama
# contains these elements.
Markdown(query_llama('what is the difference between EECS 492 and EECS 493?'))

EECS 492 and EECS 493 are both sections of the same upper-level undergraduate electrical engineering course, but there are a few important differences between them:

1. EECS 492 is typically a more advanced version of the course, designed for students who have already completed the prerequisite coursework and are looking to delve deeper into the subject matter. This section often covers more difficult and specialized topics, and may require more independent study and research.
2. EECS 493, on the other when compared with 492, is a more beginner-friendly course. It covers the basics of electrical engineering, including circuit analysis and digital logic. It is often taken by students who are new to the field and want a solid foundation in the basics of electrical engineering.
3. EECS 492 and EECS 493 are often taken together in the same semester as the two courses are complementary to one another and build upon each other. The 493 course usually lays the foundation for 492.
4. EECS 492 is more research-oriented, where as EECS 493 is more project-oriented.

In summary, EECS 492 is a more advanced version of the course and is often taken by students who have already completed the prerequisite coursework and are looking to delve deeper into the subject matter, while EECS 493 is a beginner-friendly course that covers the basics of electrical engineering.

In [44]:
grader.check("q04_01")

q04_01 results: All test cases passed!

Now, we can call `query_llama`! Run the cell below.

In [45]:
Markdown(query_llama('Tell me about EECS 485 at Michigan, but keep it concise: just one paragraph.'))

EECS 485 at the University of Michigan is a senior-level design project course that requires students to design and implement a complete embedded control system. The course is project-based and provides students with the opportunity to apply their knowledge of computer engineering principles to real-world problems. Students work in teams to design and develop a system, and are required to present their work to the class. The course also includes lectures and discussions on topics such as programming languages, compilers, and computer architecture.

To experiment:
- Run the cell many times. You'll notice that the response is very different every time – and it's almost never accurate! (Click [here](https://eecs485.org) to see what EECS 485 here is actually about.)
- Uncomment the line that says `temperature=0` in your definition of `query_llama`, and then run the above cell many times again. What do you notice now? (To see what argument is doing, go to the [documentation](https://console.groq.com/docs/api-reference#chat-create) and search for "temperature".) Recomment out the line before proceeding.
- If you remove "but keep it concise: just one paragraph.", what do you notice?

Now we have a way of passing queries to a Large Language Model and getting back results. Right now, it's not knowledgeable enough to answer questions about EECS classes. Soon, we'll change that.

We'll get back to using `query_llama` in the final part of this question. For now, we need to switch our attention to implementing RAG – that is, being able to find the syllabus documents that are most similar to our input query. Once we implement it, when we pass our (new) function the input `'Tell me about EECS 485 at Michigan, but keep it concise: just one paragraph.'`, it'll provide accurate, up-to-date information about EECS 485, since we'll send the syllabus for EECS 485 to Llama along with the original input. **Keep this goal in mind. The next few parts may seem unrelated, but they all come together at the end!**

### Question 4.2 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">1 Point</div>

A **token** is an alphanumeric string. In Lecture 12, we referred to tokens as "terms". Before computing any numbers, we need to find the terms in each syllabus, i.e. we need to **tokenize** each syllabus.

Complete the implementation of the function `tokenize`, which takes in a string (`string`) of text and returns a list containing all of the tokens in `string`. Convert all characters to lowercase before extracting tokens.

Example behavior is given below.

```python
>>> tokenize("EECS 398-003 Practical Data Science's about data management and applied machine learning.")
['eecs',
 '398',
 '003',
 'practical',
 'data',
 'science',
 's',
 'about',
 'data',
 'management',
 'and',
 'applied',
 'machine',
 'learning']

>>> tokenize(open('data/syllabi/485.txt').read())[:20]
['eecs',
 '485',
 'web',
 'systems',
 'syllabus',
 'the',
 'university',
 'of',
 'michigan',
 'fall',
 '2024',
 'a',
 'holistic',
 'course',
 'of',
 'modern',
 'web',
 'systems',
 'and',
 'technologies']
```

Note that this part is only worth 1 point, so it shouldn't take very long!

In [46]:
def tokenize(string):
    new = re.findall(r'\w+',string.lower())
    return new

# Feel free to change the input below to test out your implementation of tokenize.
tokenize("EECS 398-003 Practical Data Science's about data management and applied machine learning.")

['eecs',
 '398',
 '003',
 'practical',
 'data',
 'science',
 's',
 'about',
 'data',
 'management',
 'and',
 'applied',
 'machine',
 'learning']

In [47]:
grader.check("q04_02")

q04_02 results: All test cases passed!

Before we move onto Question 4.3, it's worth mentioning that in practice, we'd put a bit more care into tokenizing our documents. For one, we might **lemmatize** our tokens, which would allow us to group words like `'eating'`, `'ate'`, and `'eatery'` all to `'eat'`. We've omitted such steps here for simplicity.

### Question 4.3 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">4 Points</div>

Complete the implementation of the function `files_to_bow`, which takes in a string describing the **path** to a folder with syllabus files (`path`) and returns the corresponding **bag of words matrix** as a DataFrame, with:
- One row per file, indexed by the file name. The DataFrame should be sorted by the index in ascending order.
- One column per unique word (token) among all syllabi (i.e. across the entire corpus). The order of the columns in the DataFrame does not matter.
- Values corresponding to the number of occurrences of each word in each file.

Example behavior is given below.

```python
>>> out = files_to_bow('data/syllabi')
>>> out.shape
(29, 4306)

>>> out.loc['280.txt', 'computer']
12
```

Some guidance:
- You must implement all of the steps by hand, i.e. no using `sklearn`'s `CountVectorizer`.
- To find all of the files in a folder, use `os.listdir` (we've already imported `os`). Make sure to verify that the files you're processing end in `.txt` – there may be other files in `path` that aren't valid syllabi, and we don't want to process those.
- Our solution involved creating an intermediate helper function that read in the necessary files, tokenized them, and stored them in an appropriate data structure. You can design your implementation however you'd like, but it's a good idea to break it down into smaller pieces.
- Since we've already tokenized each file, it's not necessary to use regular expressions to count the number of occurrences of particular words in each document. Look into the list `count` method, which you can use in conjunction with a `for`-loop or the Series `apply` method. Our solution follows the work in Lecture 12 closely.
- Our solution only takes ~5 seconds to run on `files_to_bow('data/syllabi')`. Make sure yours is similarly quick.

In [48]:
def files_to_bow(path):
    name = []
    text = []
    
    for i in os.listdir(path):
        if i.endswith(".txt"):  
            file_path = os.path.join(path, i)
            with open(file_path, "r") as f:
                content = f.read()
            name.append(i)
            text.append(tokenize(content))
    

    new = pd.DataFrame({"filename": name, "text": text})
    new.set_index('filename', inplace=True)
    new = new.sort_index()
    name = sorted(name)

    all_words = new['text'].explode().value_counts().index
    
    counts_dict = {fname: {} for fname in name}
    
    for term in (all_words):
        for i in range(len(new)):
            filename = name[i]
            counts_dict[filename][term] = new['text'].iloc[i].count(term)
    
    counts = pd.DataFrame.from_dict(counts_dict, orient='index').fillna(0).astype(int)

    return counts.sort_index()

make = files_to_bow('data/syllabi')
make 

,the,to,and,you,a,of,in,for,will,be,...,1017,012,ereader,1006,opinion,011,auditorium,chesebrough,flaws,withdrawal
183.txt,183,131,97,101,77,74,70,76,47,43,...,0,0,0,0,0,0,0,0,0,0
203.txt,206,136,105,114,59,68,80,65,62,50,...,0,0,0,0,0,0,0,0,0,0
270.txt,57,25,26,31,34,22,19,12,21,26,...,0,0,1,0,1,0,0,0,1,0
280.txt,157,102,128,77,94,73,70,78,20,32,...,0,0,0,0,0,0,0,0,0,0
281.txt,214,132,78,110,110,85,83,63,61,64,...,0,0,0,0,0,0,0,0,0,1
370.txt,114,76,52,66,62,45,50,44,50,53,...,0,0,0,0,0,0,0,0,0,0
373.txt,63,37,46,22,26,31,20,13,16,18,...,0,0,0,0,0,0,0,0,0,0
376.txt,116,94,90,65,59,60,45,56,45,38,...,0,0,0,0,0,0,0,0,0,0
388.txt,14,6,18,7,1,18,3,8,4,3,...,0,0,0,0,0,0,0,0,0,0
390.txt,188,115,102,78,85,87,63,82,39,30,...,0,0,0,0,0,0,0,0,0,0


In [49]:
grader.check("q04_03")

q04_03 results: All test cases passed!

Since we'll need it in all of our future calculations, we'll create a globally-defined instance of `bow` below. **Make sure that throughout the rest of your notebook, `bow` is defined exactly as below!**

In [50]:
bow = files_to_bow('data/syllabi')
bow.head()

,the,to,and,you,a,of,in,for,will,be,...,1017,012,ereader,1006,opinion,011,auditorium,chesebrough,flaws,withdrawal
183.txt,183,131,97,101,77,74,70,76,47,43,...,0,0,0,0,0,0,0,0,0,0
203.txt,206,136,105,114,59,68,80,65,62,50,...,0,0,0,0,0,0,0,0,0,0
270.txt,57,25,26,31,34,22,19,12,21,26,...,0,0,1,0,1,0,0,0,1,0
280.txt,157,102,128,77,94,73,70,78,20,32,...,0,0,0,0,0,0,0,0,0,0
281.txt,214,132,78,110,110,85,83,63,61,64,...,0,0,0,0,0,0,0,0,0,1


### Question 4.4 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Complete the implementation of the function `bow_to_tfidf`, which takes in a bag of words matrix (`bow`) returned by `files_to_bow`. `bow_to_tfidf` should return a DataFrame with the same row labels and column labels as `bow`, but with all values converted to TF-IDFs – that is, the outputted DataFrame should contain the TF-IDF of every word in every file.

Example behavior is given below.

```python
# Here, we're referring to the globally-defined bow.
>>> out = bow_to_tfidf(bow)
>>> out.shape == bow.shape
True

>>> out.loc['485.txt', 'science']
0.0005272966438261625
```

Some guidance:
- Follow our logic from Lecture 12 to convert `bow` to a TF-IDF matrix. Your implementation here should be relatively short (< 10 lines).
- While not strictly required (in that we won't test it), we recommend you implement `compute_idfs`, which takes in a DataFrame like `bow` and returns a **Series** containing the inverse document frequency (IDF) of each word in `bow`. Not only will this help compartmentalize your work for this question, but it'll make your life much easier in Question 4.5, when you'll again need to use the IDFs of every word in the corpus.

In [51]:
def compute_idfs(bow):
    num_docs = len(bow)
    doc_freqs = bow.astype(bool).sum()
    idfs = np.log(num_docs / (doc_freqs))
    return idfs

def bow_to_tfidf(bow):
    idfs = compute_idfs(bow)
    tf = bow.div(bow.sum(axis=1), axis=0)
    tfidfs = tf * idfs
    
    return tfidfs

# Uncomment the line below once you've implemented bow_to_tfidf.
bow_to_tfidf(bow).head()

,the,to,and,you,a,of,in,for,will,be,...,1017,012,ereader,1006,opinion,011,auditorium,chesebrough,flaws,withdrawal
183.txt,0.0,0.0,0.0,0.000813,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
203.txt,0.0,0.0,0.0,0.000925,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
270.txt,0.0,0.0,0.0,0.000948,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.002933,0.0,0.002933,0.0,0.0,0.0,0.002933,0.000000
280.txt,0.0,0.0,0.0,0.000631,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
281.txt,0.0,0.0,0.0,0.000852,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000743


In [52]:
grader.check("q04_04")

q04_04 results: All test cases passed!

Before we move forward, it's worth stopping and looking at what we've already accomplished. Run the cell below to see the 5 words with the highest TF-IDFs in each syllabus.

In [53]:
def five_largest(row):
    return ', '.join(row.index[row.argsort()][-5:])

bow_to_tfidf(bow).apply(five_largest, axis=1)

183.txt               zybooks, eecs183, codelab, 183, ecoach
203.txt             match, enrolled, homework, ecoach, pages
270.txt                   digital, closed, lab, really, book
280.txt                     partners, 280, lab, fci, debrief
281.txt            999999, least, attempt, eecs281admin, lab
370.txt                   score, hardware, computers, 55, 99
373.txt          373, extenuating, penalty, embedded, unfair
376.txt         algorithm, solution, solvable, 376, workshop
388.txt       asynchronously, musaddequr, br, security, face
390.txt         score, paradigms, partners, 390, partnership
445.txt                dow, theoretical, 30pm, machine, zhao
453.txt         deep, unsupervised, 09, learning, supervised
465.txt           standing, 2004, berenson, press, cambridge
467.txt           robots, localization, robot, physical, sum
470.txt         projects, format, verilog, architecture, 470
471.txt             cuda, ia, graphics, processors, parallel
473.txt                 

Compare that to the 5 words with the highest frequences in each syllabus:

In [54]:
bow.apply(five_largest, axis=1)

183.txt                  a, and, you, to, the
203.txt                 in, and, you, to, the
270.txt                  be, and, you, a, the
280.txt                  for, a, to, and, the
281.txt                   of, a, you, to, the
370.txt                   be, a, you, to, the
373.txt                   a, of, to, and, the
376.txt                 of, you, and, to, the
388.txt               for, the, face, of, and
390.txt                  of, we, and, to, the
445.txt                  to, a, will, of, the
453.txt    1, and, supervised, week, learning
465.txt              of, course, be, the, and
467.txt                   is, of, a, the, and
470.txt                and, be, to, will, the
471.txt                  in, of, you, to, the
473.txt                be, will, to, and, the
475.txt                eecs, is, the, of, and
481.txt                  are, a, to, and, the
482.txt                  a, and, you, to, the
483.txt               code, you, and, to, the
484.txt          will, course, and

Hopefully, the value of TF-IDF is clear, but it's also clear that TF-IDF isn't perfect in summarizing documents. But, as we'll soon see, it'll serve our purposes well!

Before you move to Question 4.5, there's one piece of syntax that you'll find useful: the Series `reindex` method. Here's an example of how it works:

In [55]:
things = pd.Series({'a': 2, 'b': 5, 'c': 1})
things

a    2
b    5
c    1
dtype: int64

In [56]:
stuff = pd.Series({'a': 'hello', 'b': 'hi', 'x': 9})
stuff

a    hello
b       hi
x        9
dtype: object

In [57]:
things.reindex(stuff.index)

a    2.0
b    5.0
x    NaN
dtype: float64

In [58]:
things.reindex(stuff.index).fillna(0)

a    2.0
b    5.0
x    0.0
dtype: float64

### Question 4.5 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Complete the implementation of the function `new_query_to_tfidf`, which takes in a string (`query_string`) and a bag of words matrix (`bow`) and returns a Series such that:
- The index contains the same labels as `bow`'s columns (meaning that if `bow` has 4306 columns, the outputted Series should have 4306 elements).
- The values contain the TF-IDF of each word, using `query_string` to compute TFs and **the entire corpus of syllabi (not including the new query)** to compute IDFs.

Example behavior is given below.

```python
>>> out = new_query_to_tfidf('yooo I am very very very interested in a practical machine learning course', bow)
>>> out.shape
(4306,)

# Most of the values in out are 0, since
# "yooo I am very very very interested in a practical machine learning course"
# doesn't contain most of the 4306 words in bow.
# Since 'yooo' is not in bow.columns, it doesn't appear in the index of out, either.
>>> out[out > 0]
machine       0.090005
interested    0.174514
very          0.328012
am            0.152385
i             0.032527
practical     0.109337
learning      0.050711
dtype: float64
```

To be clear, the TF-IDF of a word $t$ in a new query string $q$ is:

$$\text{tfidf}(t, q) = \underbrace{\frac{\text{\# of occurrences of $t$ in $q$}}{\text{total \# of tokens in $q$}}}_{\text{computed using } q \: (\texttt{query\_string})} \cdot \underbrace{\log \left(\frac{\text{total \# of syllabi}}{\text{\# of syllabi in which $t$ appears}} \right)}_{\text{computed solely using \texttt{bow}}}$$

Note that this means that the IDFs of each word have nothing to do with the `query_string` that is passed in. This is precisely why we suggested you implement `compute_idfs(bow)` in the previous part – because it would help your implementation of `bow_to_tfidf`, and also help your implementation of `new_query_to_tfidf`.

Some additional guidance:
- This function should only take a few lines to implement, but requires combining several steps, going all the way back to Question 4.2. Think about how the `reindex` method might be useful.
- In the function signature below, you'll see `new_query_to_tfidf(query_string, bow=bow)`. `bow=bow` sets the default value of the `bow` argument to the globally-defined value of `bow`, meaning if we only pass one argument (`query_string`) to `new_query_to_tfidf`, it will automatically use the global `bow`. It's important for our function to be able to take in bag of words matrices other than our globally-defined `bow`, in case we want to use it on a different corpus of documents. But, most of the time we will call it on the global `bow`, so this is done for convenience.

In [59]:
def new_query_to_tfidf(query_string, bow=bow):
    query_tokens = tokenize(query_string)
    query_counts = pd.Series(query_tokens).value_counts()
    set_1 = set(compute_idfs(bow).index)
    set_2 = set(query_counts.index)
    diff = set_2 - set_1
    new = (query_counts/len(query_tokens)) * (compute_idfs(bow))
    return new.drop(index = diff).fillna(0)

# Feel free to change the input below to test out your implementation of new_query_to_tfidf.
out = new_query_to_tfidf('yooo I am very very very interested in a practical machine learning course')
out

0              0.0
00             0.0
000            0.0
001            0.0
002            0.0
              ... 
zhongren       0.0
zhongsydney    0.0
zone           0.0
zoom           0.0
zybooks        0.0
Length: 4306, dtype: float64

In [60]:
def new_query_to_tfidf(query_string, bow=bow):
    # Tokenize the query string
    query_tokens = tokenize(query_string)
    
    # Count the occurrences of each token
    query_counts = pd.Series(query_tokens).value_counts()

    # Calculate term frequency (TF)
    tf = query_counts / len(query_tokens)
    
    # Compute IDF only for terms that exist in the bow
    idfs = compute_idfs(bow)

    # Keep only the terms that exist in the query and in the bow
    valid_terms = tf.index.intersection(idfs.index)

    # Calculate TF-IDF for the valid terms
    tfidf = tf[valid_terms] * idfs[valid_terms]

    # Return the resulting Series, filled with 0s for terms not present
    return tfidf.reindex(bow.columns, fill_value=0)

out = new_query_to_tfidf('yooo I am very very very interested in a practical machine learning course')
out

the            0.0
to             0.0
and            0.0
you            0.0
a              0.0
              ... 
011            0.0
auditorium     0.0
chesebrough    0.0
flaws          0.0
withdrawal     0.0
Length: 4306, dtype: float64

In [61]:
grader.check("q04_05")

q04_05 results: All test cases passed!

Let's take stock of what we have so far.
- We have the TF-IDFs of every word in every document in our corpus. This means that we have a **vector representation** of each syllabus.
- We have a function that can take any query string and turn it into a **vector** of TF-IDF scores, as well.

Now, we can use techniques from Lecture 12 – specifically, cosine similarity – to find the syllabi that are most similar (and, hence, most relevant) to our query string!

### Question 4.6 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Complete the implementation of the function `top_n_similar_documents`, which takes in a string (`query_string`), a positive integer `n`, and a bag of words matrix (`bow`) and returns a list containing the names of the `n` most similar documents to `query_string`. 

Use cosine similarity to measure the similarity between two vectors; you can implement cosine similarity however you'd like. Remember that document names are stored in the index of `bow`. The documents in the returned list should be sorted in **decreasing order of similarity**.

Example behavior is given below.

```python
>>> top_n_similar_documents('yooo I am very very very interested in a practical machine learning course', 3, bow)
['467.txt', '445.txt', '453.txt']

>>> top_n_similar_documents('C++ programming and systems design', 4, bow)
['482.txt', '473.txt', '370.txt', '281.txt']
```

In [62]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [63]:
def top_n_similar_documents(query_string, n, bow=bow):
    query_tfidf = new_query_to_tfidf(query_string, bow).fillna(0)
    arr =  np.array(query_tfidf)
    tfidf = bow_to_tfidf(bow).fillna(0)
    new = [0] * len(tfidf)
    for i in range(len(tfidf)):
        vec = np.array(tfidf.iloc[i])
        new[i] = cosine_similarity(vec, arr)
    npnew = pd.Series(new)
    yay = npnew.nlargest(n).index
    return  [bow.index[i] for i in yay]


# Feel free to change the inputs below to test out your implementation of top_n_similar_documents.
top_n_similar_documents('hey teach me about programming', 3)

['471.txt', '485.txt', '482.txt']

In [64]:
grader.check("q04_06")

q04_06 results: All test cases passed!

Awesome! You've implemented the retrieval step in RAG. That is, given a query, you're able to automatically find the most relevant documents in our "knowledge database" for answering that query.

It's time for the final step: passing a `query_string`, along with the contents of the most relevant documents, to a Large Language Model (which we already learned how to access, using `query_llama`).

### Question 4.7 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">4 Points</div>

Complete the implementation of the function `ask_gpteecs`, which takes in a string (`query_string`) containing a question about EECS courses, a positive integer `n`, and a bag of words matrix `bow`. `ask_gpteecs` should return a **string** containing the result of:

- querying Llama 3 using `query_llama` from Question 4.1,
- where the query contains **both** the contents of `query_string` and
- the **top `n`** most similar syllabus documents,
- stitched together in a way that you deem appropriate.

Here's what we mean by "in a way that you deem appropriate." Suppose our query is `'yooo I am very very very interested in a practical machine learning course'`, and suppose `n=3` (the default).
- The top 3 most similar documents are `'467.txt'`, `'445.txt'`, and `'453.txt'`.
- If we just ask Llama, `'yooo I am very very very interested in a practical machine learning course'`, it won't know anything about EECS 467, EECS 445, or EECS 453. If we ask it, `'yooo I am very very very interested in a practical machine learning course, tell me about them: 467.txt, 445.txt, and 453.txt'`, it also won't know anything about those courses.
- Instead, once we identify which (3) documents are most relevant, we need to read them in as strings once again using `open`, then create a new `query_string` that looks something like:

```python
'''
Hi! I'm looking to answer this query that a student sent me, regarding EECS courses at the University of Michigan:

yooo I am very very very interested in a practical machine learning course

Here are some relevant courses from my knowledge base.

here's EECS 467
EECS 467: Autonomous Robots
Software methods and implementation for robot perception, world mapping, ...
...

here's EECS 445
Syllabus
Introduction to Machine LearningFall 2016
The course is a programming-focused introduction to Machine Learning.
...

here's EECS 453
Course Instructor: Prof. Qing Qu
Course Time: Mon/Wed 12:00 PM – 1:30 PM
...
'''
```

- You can structure your final query string however you'd like, and you're encouraged to experiment with different phrasings to see if they influence your results; you can start by copying the example format above, but then try and make it your own. (This is called **prompt engineering**.)
- In the example above, we only included the first few lines of the relevant syllabi, but in your actual prompts, you'd include the entire text. You'll need to figure out a way of programmatically adding the course numbers and course syllabi text to your prompt string – remember, `n` might be something other than 3.

In [65]:
def ask_gpteecs(query_string, n=3, bow=bow):
    similar = top_n_similar_documents(query_string, n, bow)
    arr = []
    content = []
    names = []
    new = f''' Here's what we mean by "in a way that you deem appropriate." Suppose our query is
    "{query_string}".
    
    Here are the top {n} syllabi that are most similar to your query:
    '''
    hello = query_llama(new)
    for i in similar:
        open_file = open(f'data/syllabi/{i}', 'r')
        #content.append(open_file.read())
        #names.append('EECS ' + i[:3])
        hello += f'EECS {i[:3]}: {open_file.read()}'

    return hello
    
    

# Feel free to change the inputs below to test out your implementation of ask_gpteecs.
# The Markdown function behaves like the print function,
# but renders text formatting (e.g. bolding, bullet points) when the output from Llama
# contains these elements.
(Markdown(ask_gpteecs('teach me about machine learning')))
#outs = [ask_gpteecs('yooo I am very very very interested in a practical machine learning course') for _ in range(1)]
#outs

1. Introduction to Machine Learning (6.0 units)
    
    - Syllabus: Introductory guide to ML concepts and techniques, including supervised learning, unsupervised learning, and reinforcement learning. Covers common ML algorithms like linear regression, logistic regression, decision trees, and neural networks.
    - Prerequisites: Basic programming skills, calculus, probability, and linear algebra.
    - Learning outcomes:
        - Understand the principles of ML and the various techniques used in data analysis and modeling.
        - Develop models using common ML algorithms and evaluate their performance using metrics like accuracy and precision.
        - Implement and compare different ML models on real-world datasets.
    
    - Contact hours: 72 hours (12 2-hour sessions)
    - Course materials: Lectures, assignments, exams, and practical exercises.
    - Assessment: Graded assignments, exams, and a final project.
    
 2. Advanced Machine Learning (6.0 units)
    
    - Syllabus: Deep learning, natural language processing, computer vision, and reinforcement learning. Covers advanced techniques for dealing with complex data structures and large-scale datasets.
    - Prerequisites: Basic understanding of ML, programming skills, and comfort with mathematical concepts and linear algebra.
    - Learning outcomes:
        - Gain advanced knowledge in ML, focusing on advanced techniques and modern applications.
        - Develop models using cutting-edge ML techniques like deep learning, NLP, and computer vision.
        - Apply advanced ML methods to solve complex problems and tackle real-world challenges.
    
    - Contact hours: 72 hours (12 2-hour sessions)
    - Course materials: Lectures, assignments, research papers, and real-world case studies.
    - Assessment: Graded assignments, exams, research paper presentation, and a final project.
    
3. Data Science for Machine Learning (6.0 units)
    
    - Syllabus: Data preprocessing, feature engineering, model evaluation, and dealing with imbalanced datasets. Focuses on practical skills required to be a data scientist in the context of ML.
    - Prerequisites: Basic understanding of ML, programming skills, and experience with Python or R.
    - Learning outcomes:
        - Develop models using ML techniques in a way that is consistent with your field of study and work experience.
        - Apply data preparation, feature engineering, and model evaluation techniques to real-world datasets.
        - Identify common problems in data science and ML and propose potential solutions.
    
    - Contact hours: 72 hours (12 2-hour sessions)
    - Course materials: Lectures, assignments, real-world case studies, and online resources/toolsets (Python or R).
    - Assessment: Graded assignments, exams, group projects, and a final project for real-world datasets.
    
Each of these courses aims to ensure you acquireEECS 453: Course Instructor: Prof. Qing Qu

Course Time: Mon/Wed 12:00 PM – 1:30 PM,  3 credit hour

Office Hour: Wed 3:30 PM – 5:00 PM

Prerequisite: EECS 351, or EECS 301, or any linear algebra courses

Notice: This is an entry-level ECE machine learning course targeted for senior EE & CE undergraduate, and junior master students outside SIPML area. All students outside EECS that want to learn the basics of ML are also welcome! Compared to EECS 445, this course places slightly greater emphasis on mathematical principles and is better suited for students who have limited experience with programming and machine learning. 

Overview: The class will cover basic principles in machine learning, such as unsupervised learning (e.g., clustering, mixture models, dimension reduction), and supervised learning (e.g., regression, classification, neural networks & deep learning). For each topic, key algorithmic ideas/intuitions and basic theoretical insights will be highlighted.

Course Materials: slides and videos will be accessed via Canvas (TBA). Tentative topics that will be covered in this course are supervised learning, unsupervised learning, and reinforcement learning:

Basics of probability, linear algebra, and optimization
Regression and linear prediction
Support vector machines and kernel methods
Deep neural networks
Dimension reduction: PCA, autoencoder
Clustering (Kmeans, Mixture of Gaussians, EM)
Representation learning: nonnegative matrix factorization, dictionary learning
Assessment: (i) 5 homework assignments (40%), (ii) mid-term exam (30%), (iii) course projects (25%), (iv) participation & course evaluation (5%)

Assessment	 Percentage  
Homework (5)	40%
Midterm Exam	30%
Projects	25%
Participation & Course Evaluation	5%
Textbook: We recommend the following books and articles, although we will not follow them closely.

Foundations of Machine Learning, by Mehryar Mohri, Afshin Rostamizadeh, and Ameet Talwalkar.
The Elements of Statistical Learning: Data Mining, Inference, and Prediction, by Trevor Hastie, Robert Tibshirani, and Jerome Friedman.
Deep Learning, by Ian Goodfellow, Yoshua Bengio, and Aaron Courville.
Mathematics for Machine Learning, by Marc Peter Deisenroth, A. Aldo Faisal, and Cheng Soon Ong.
Linear Algebra and Optimization for Machine Learning, by Charu C. Aggarwal.
Related courses:

EECS 445. Introduction to Machine Learning
EECS 453. Applied Matrix Algorithms for Signal Processing, Data Analysis, and Machine Learning
EECS 505. Computational Data Science and Machine Learning
EECS 545. Machine Learning
Course Syllabus (Note: the schedule is tentative, and is subject to change during the semester.)

Week	Date	 Topic	Contents	Homework, Review
Week-1-1	08/29	Introduction (Remote)	Course overview	 
Week-1-2	08/31	Supervised Learning (Remote)	Introduction to supervised learning, linear models, regularization	Linear Algebra Review 
Week-2-1	09/05	Labor Day	No class	 
Week-2-2	09/07	Supervised Learning	Learning Theory	Probability Review, HW1 Release 
Week-3-1	09/12	Supervised Learning	Linear regression I	 
Week-3-2	09/14	Supervised Learning	Linear regression II	Python Review 
Week-4-1	09/19	Supervised Learning	Linear Classifiers	 
Week-4-2	09/21	Supervised Learning	Linear Discriminant Analysis	HW1 Due, HW2 Release 
Week-5-1	09/26	Supervised Learning (remote)	Logistic regression	 
Week-5-2	09/28	Supervised Learning (remote)	Optimization methods I	 
Week-6-1	10/03	Supervised Learning	Optimization methods II	 
Week-6-2	10/05	Supervised Learning 	Support vector machine (SVM) I	HW2 Due,  HW3 Release  
Week-7-1	10/10	Supervised Learning 	Support vector machine (SVM) II	 
Week-7-2	10/12	Supervised Learning 	Support vector machine (SVM) III	 
Week-8-1	10/17	Fall Study Day	No class	 
Week-8-2	10/19	Supervised Learning	Dual SVM	 HW3 Due
Week-9-1	10/24	Supervised Learning	Nonlinear models, kernel methods	 
Week-9-2	10/26	Supervised Learning	Introduction to deep neural networks I	 
Week-10-1	10/31	Supervised Learning	Introduction to deep neural networks II	 
Week-10-2	11/02	Supervised Learning	Introduction to deep neural networks III	 
Week-11-1	11/07	Midterm	Midterm	 
Week-11-2	11/09	Unsupervised Learning	Introduction to unsupervised learning, clustering problem, K-means	Project Proposal Due, HW4 Release
Week-12-1	11/14	Unsupervised Learning	K-means, mixtures of Gaussian, expectation maximization	 
Week-12-2	11/16	Unsupervised Learning	Dimension reduction, PCA	 
Week-13-1	11/21	Unsupervised Learning	Dimension reduction II	 
Week-13-2	11/23	Thanksgiving	No Class	 
Week-14-1	11/28	Unsupervised Learning (Remote)	Representation learning, matrix factorization	HW4 Due, HW5 Release   
Week-14-2	11/30	Unsupervised Learning (Remote)	Autoencoder & self-supervised learning	 
Week-15-1	 12/05	Unsupervised Learning	Generative Models	HW5 Due 
Week-15-2	 12/07	 Final Presentation	EECS 445: Syllabus
Introduction to Machine LearningFall 2016
The course is a programming-focused introduction to Machine Learning. Increasingly, extracting value from data is an important contributor to the global economy across a range of industries. The field of Machine Learning provides the theoretical underpinnings for data-analysis as well as more broadly for modern artificial intelligence approaches to building artificial agents that interact with data; it has had a major impact on many real-world applications.

This is an undergraduate course. Graduate students seeking to take a machine learning course should consider EECS 545.

The course will emphasize understanding the foundational algorithms and “tricks of the trade” through implementation and basic-theoretical analysis. On the implementation side, the emphasis will be on practical applications of machine learning to computer vision, data mining, speech recognition, text processing, bioinformatics, and robot perception and control. Real data sets will be used whenever feasible to encourage understanding of practical issues. The course will provide an opportunity for an open-ended research project. On the theoretical side, the course will give a undergraduate-level introduction to the foundations of machine learning topics including regression, classification, kernel methods, regularization, neural networks, graphical models, and unsupervised learning.

Basic Information
Professors	Dr. Jacob Abernethy and Dr. Jia Deng
TAs	Chansoo Lee, Zhao Fu, Ben Bray, Valli Chockalingham
Prerequisites	
The only enforced prerequisite is EECS 281. However, if you are not familiar with at least one of the following topics you will struggle in this course.

Linear algebra at the level of MATH 419 or MATH 214, but preferrably at the level of MATH 217.
Probability at the level of EECS 401, MATH 425, or equivalent.
We understand that most EECS students will only ever encounter a proper subset of these topics, so we will be providing a brief review of important concepts at the beginning of the semester. We'll move fast, though, so be warned!

Textbook	
A good textbook is invaluable for self-study. Although there is no required textbook, we highly recommend purchasing one of the following books for your own use and future reference:

Murphy, Kevin P. Machine learning: A Probabilistic Perspective. MIT press, 2012.
Bishop, Christopher M. Pattern Recognition and Machine Learning. Springer-Verlag New York, Inc., 2006.
Lectures	
This semester, we will be experimenting with a flipped classroom format. The later section will be a typical lecture and will be video recorded for online viewing. The earlier section will be more hands-on, where students will spend most of their time working together in groups to better understand challenging concepts.

Section 001 (Hands-on Lecture) Mon/Wed 4:30-6pm, 1670 BBB
Section 002 (Standard Lecture) Mon/Wed 6-7:30pm, Chesebrough Auditorium
Discussion	
Discussions begin Tuesday, September 13, 2016.

Discussion 011 (Ben) Fri 11:30am-12:30pm, 1006 DOW
Discussion 012 (Zhao) Thu 4:30-5:30pm, 1017 DOW
Discussion 013 (Zhao) Fri 1:30-2:30pm, 1303 EECS
Discussion 014 (Chansoo) Tue 4:30-5:30pm, 2150 DOW
Discussion 016 (Valli) Thu 2:30-3:30pm, 1005 EECS
Office Hours	See the course calendar below for our office hour schedule.
Communication	No email policy! Use Piazza instead! Our only exception to this policy is for personal issues, for which you may email or make an appointment with a professor.
EECS 471: Course Overview
The goal of this class is to teach parallel computing and developing applications for massively parallel processors (e.g. GPUs). Self­driving cars, machine learning and augmented reality are examples of applications involving parallel computing. The class focuses on computational thinking, forms of parallelism, programming models, mapping computations to parallel hardware, efficient data structures, paradigms for efficient parallel algorithms, and application case studies.

The course will cover popular programming interface for graphics processors (CUDA for NVIDIA processors), internal architecture of graphics processors and how it impacts performance, and implementations of parallel algorithms on graphics processors. The curriculum will be delivered in ~29 lectures. The class has heavy programming components, including five hands­on assignments and a final project.

Prerequisites
Students must have taken both EECS 281 and EECS 370

UG Requirements met by the class:
4 credits, Upper-level elective for CS and CE majors, Flex Tech electives.

Programming Assignments
Six assignments will be assigned during the term.

The most common reason for not doing well on the assignments is not starting them early enough. You will be given plenty of time to complete each assignment. However, if you wait until the last minute to start, you may not be able to finish. Plan to have it finished about 2 days ahead of the due date - many unexpected problems arise during programming, especially in the debugging phase. Plan for these things to happen. Your lack of starting early is not an excuse for turning in your assignment late, even if some unfortunate situations arise such as having your computer crash.

There are many sources of help on which you can draw. Many questions can be submitted to the course staff and your fellow classmates via the class forum. The policies for using the class forum are contained in the first post in the forum. These will typically be answered within the day, often more quickly during working hours. However, some types of questions cannot be answered without seeing your code. If you have detailed questions on your program, speak to a IA or professor in office hours.

Students are also encouraged to help one another on the course concepts (but not the implementation of the assignments). One of the best ways for you to make sure that you understand a concept is to explain it to someone else. Keep in mind, however, that you should not expect anyone else to do any part of your programming assignment for you. The programming assignment that you turn in must be completely your own.

Turning in Assignments
Assignments are due at 11:59 pm exactly on the due date.

Each assignment has a programming component to be submitted on Great Lakes, and a quiz component to be submitted on Gradescope. The programming component is 80% of the assignment grade, and the quiz component is 20% of the assignment grade.

Final Project
There will be a final project with an open ended implementation. You will be responsible for using the knowledge you gained throughout the course to optimize an applied problem. Project details will be posted later in the term.

Assignment and Project Grading
The assignments and project will be graded on both correctness and performance. All grading questions should first be discussed with your IA. If you cannot resolve a problem with the IA, bring the project to the instructor.

Doing Your Own Assignments
All assignments and exams in this course are to be done on your own, with the exception of the final project where you will be allowed to have a partner. Any suspected violation will result in the initiation of formal procedures with the LS&A or Engineering Honor Council. Violators will receive a 0 in the assignment, in addition to additional grade repercussions, as recommended by the appropriate Honor Council.

We will be using a sophisticated automated program to correlate programming work, including those submitted in previous semesters.

We do encourage students to help each other learn the course material. As in most courses, there is a boundary separating these two situations. You may give or receive help on any of the concepts covered in lecture or discussion and on the specifics of CUDA syntax. You are allowed to consult with other students in the current class to help you understand the assignment specification (i.e. the problem definition). However, you may not collaborate in any way when constructing your solution - the solution to the assignment must be generated by your work alone and the work of other students must not have contributed to your solution. You are not allowed to work out the programming details of the problems with anyone or to collaborate to the extent that your programs are identifiably similar. You are not allowed to look at or in any way derive advantage from the existence of specifications or solutions prepared in prior years (e.g. programs written by former students, solutions provided by instructors, or handouts).

If you have any questions as to what constitutes unacceptable collaboration, please talk to the instructor right away. You are expected to exercise reasonable precautions in protecting your own work. Do not leave your program in a publicly accessible directory, and take care when discarding printouts.

Exams
There will be two exams this semester. You are expected to take the exams at the scheduled times. If you do not take an exam without verifying a documented medical or personal emergency causing you to miss an exam, you will receive a zero for that exam. If you anticipate conflicts with the exam time, declare your conflicts by the date specifed on the exams page. The exam dates are given near the beginning of the semester so you can avoid scheduling job interviews or other commitments on exam days. Outside commitments are not considered a valid reason for missing an exam. If you need to request any special accommodation during any exam, please bring the necessary paperwork from the SSD office by the end of the first month of classes.

Grading Policy
Final grades will be based on the total points earned on homeworks, programming assignments and exams. Factors such as class participation may be used to adjust your final grade, especially if it falls on a borderline. T We consider the following grading policies:

Programming Assignments = 50%
Final Project = 15%
Midterm = 15%
Final exam = 20%
Incompletes will generally not be given. According to university policy, doing poorly in a course is not a valid reason for an incomplete. If you are having problems in the course, your best bet is to come talk to the instructor as soon as you are aware of them.

In [66]:
grader.check("q04_07")

q04_07 results: All test cases passed!

**Great work!** You've now implemented Retrieval-Augmented Generation, and have your very own ChatGPT-like interface that knows about EECS classes.

Unfortunately, it's not perfect. Look what happens with the following query:

In [67]:
#Markdown(ask_gpteecs('what is different about eecs 280 and eecs 281'))

At the bottom, the error says:

```
BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
```

This is telling us that the query sent to Llama is longer than its **context window**, an idea we discussed at the start of Question 4. Since we're using the model `'llama3-8b-8192'`, the largest possible query we can send is 8192 tokens. If we take a look at the length of each syllabus, we see that the 280 and 281 syllabi together are longer than 8192 tokens (not including punctuation, or the input query, or our instructions):

In [68]:
bow.sum(axis=1).sort_values(ascending=False)

281.txt    4533
183.txt    4358
203.txt    4325
280.txt    4284
390.txt    4177
492.txt    3387
376.txt    3068
370.txt    2810
481.txt    2398
482.txt    2254
485.txt    2219
489.txt    2125
493.txt    1878
494.txt    1878
470.txt    1440
373.txt    1317
473.txt    1205
270.txt    1148
467.txt    1121
471.txt    1106
490.txt     875
453.txt     735
487.txt     582
445.txt     547
483.txt     540
388.txt     400
465.txt     388
475.txt     372
484.txt     246
dtype: int64

Feel free to keep toying with `query_llama` (see the [Groq documentation here](https://console.groq.com/docs/quickstart) to see what you can customize) and `ask_gpteecs` to try and improve the performance of your implementation.

## Finish Line 🏁

Congratulations! You're ready to submit Homework 6.

To submit your homework:

1. Select `Kernel -> Restart & Run All` to ensure that you have executed all cells, including the test cells.
2. Read through the notebook to make sure everything is fine and all tests passed.
3. Run the cell below to run all tests, and make sure that they all pass.
4. Download your notebook using `File -> Download as -> Notebook (.ipynb)`, then upload your notebook to Gradescope under "Homework 6".
5. Stick around while the Gradescope autograder grades your work. **Remember that Homework 6 has no hidden tests! This means the tests you see in your notebook are the exact same as the ones that will be used to grade your work on Gradescope. When you submit on Gradescope, you'll see your score shortly after you submit, once the autograder finishes running.** 
6. Check that you have a confirmation email from Gradescope and save it as proof of your submission.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [69]:
grader.check_all()

q01_01 results: All test cases passed!

q01_02 results: All test cases passed!

q01_03 results: All test cases passed!

q02_01 results: All test cases passed!

q02_02 results: All test cases passed!

q02_03 results: All test cases passed!

q02_04 results: All test cases passed!

q02_05 results: All test cases passed!

q02_06 results: All test cases passed!

q02_07 results: All test cases passed!

q02_08 results: All test cases passed!

q02_09 results: All test cases passed!

q02_10 results: All test cases passed!

q03_01 results: All test cases passed!

q03_02 results: All test cases passed!

q03_03 results: All test cases passed!

q03_04 results: All test cases passed!

q04_01 results: All test cases passed!

q04_02 results: All test cases passed!

q04_03 results: All test cases passed!

q04_04 results: All test cases passed!

q04_05 results: All test cases passed!

q04_06 results: All test cases passed!

q04_07 results: All test cases passed!